# [CPSC 322](https://github.com/GonzagaCPSC322) Data Science Algorithms
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Ensemble Learning
What are our learning objectives for this lesson?
* Introduce ensemble learning
    * Bagging
    * Random Forests

Content used in this lesson is based upon information in the following sources:
* Dr. Shawn Bowers' Data Mining notes

## Warm-up Task(s)
Clone EnsembleFun from U6-Ensemble-Learning on Github.

## Today
* Announcements
    * PA7 is due 11/20. Questions?
    * IQ8 next class on PA6 topics like Naive Bayes, classifier performance metrics, etc.
* Ensemble learning
* Project partner finder

## Ensemble Learning
Basic Idea
* Different (individual) classifiers have strengths and weaknesses
* Instead of using just one, apply multiple (different) classifiers
* Use voting to select prediction
* Can lead to better prediction results

Example
* Entropy tends to result in smaller decision trees
* Which often lead to "better" predictions ... but not guaranteed
* Instead use different attribute selection approaches, and choose "most popular" prediction (among the ensemble)

## Approaches
Types of classification used
* If same type (e.g., decision trees), ensemble is **homogeneous** (our focus)
* If different types, the ensemble is **heterogeneous**

There are many ways to generate many (e.g., 100s of) classifiers from data... either use all or select some for creating the ensemble

Techniques we'll look at
* Bagging (Bootstrap Aggregation)
* Random Forests 

Different possible voting schemes for combining ensemble results
* (Simple) Majority voting
* Weighted voting

### Bagging (Bootstrap Aggregation)
Recall: Bootstrap method performs sampling with replacement
* Given a dataset $D$ of size $|D|$ instances
* Randomly select $|D|$ instances with replacement
* Results in a training set approximately 63% the size of $|D|$ and a test set 36% the size of $|D|$
* Training set will (likely) have duplicates

Basic Idea
* Generate $k$ classifiers
* Each classifier $M_i$ is trained on $D_i$ (for $1 \leq i \leq k$)
* Where $D_i$ is a bootstrap sample

To classify an instance $X$:
* Run each classifier Mi on X to get predicted label $L_i$
* Each label $L_i$ is a vote for that label
* Use the majority label (i.e., the mode) as the prediction

### Lab Task 1
Write a bootstrap function to return a random sample of rows with
replacement:

```python
def compute_bootstrapped_sample(table):
    n = len(table)
    # np.random.randint(low, high) returns random integers from low (inclusive) to high (exclusive)
    sampled_indexes = [np.random.randint(0, n) for _ in range(n)]
    sample = [table[index] for index in sampled_indexes]
    out_of_bag_indexes = [index for index in list(range(n)) if index not in sampled_indexes]
    out_of_bag_sample = [table[index] for index in out_of_bag_indexes]
    return sample, out_of_bag_sample
```

Notes:
* [`np.random.randint(low, high)`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html) returns $n$ such that $low \leq n < high$

Note that instead of using bootstrapping for testing ...
* We are using it here to **create** the ensemble for prediction
* i.e., our classifier = set of classifiers over subsamples of original dataset
* We are not using bootstrapping for testing in this case

Some advantages of bagging (bootstrap aggregation)
* Simple idea, simple to implement
* Can help deal with overfitting and noisy data (outliers)
* Can increase accuracy by reducing variance of individual classifiers

### Random Forests
Basic Idea
* Generate many different decision trees (a "forest" of trees) ... $N$ trees

Q: What are ways we could do this?
* Use bagging (bootstrap aggregation)
* Randomly select attributes (many possible trees!)
* Use different attribute selection approaches (Entropy, GINI, ...)
* Use a subset of attributes for each tree
* And so on

Random Forests approach:
* Build each tree using bagging (so different data sample used for each tree)
* At each node, select attribute from a random subset of available attributes... subset size $F$
* Use entropy to select attribute to (split) partition on
* Select the "best" subset of random trees to use in ensemble ... $M \subset N$

Note that $N$, $M$, and $F$ are all parameters of the algorithm

### Lab Task 2
Define a python function that selects F random attributes from an attribute list

```python
def compute_random_subset(values, num_values):
    values_copy = values[:] # shallow copy
    np.random.shuffle(values_copy) # in place shuffle
    return values_copy[:num_values]
```
Notes:
* [`np.random.shuffle()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.shuffle.html) performs in-place rearrangement (permutation) of given sequence
* You could also use [`np.random.choice()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) instead

### The Random Forest Procedure
1. Divide $D$ into a test and remainder set
    * Take 1/3 for test set, 2/3 for remainder set
    * Ensure test set has same distribution of class labels as $D$ ("stratified")
    * Randomly select instances when generating test set
2. Create $N$ bootstrap samples from remainder set
    * Each results in a **training** (63%) and **validation** (36%) set
    * Build and test a classifier for each of the N bootstrap samples
    * Each classifier is a decision tree using $F$-sized random attribute subsets
    * Determine accuracy of classifier using validation set
3. Pick the $M$ best classifiers generated in step 2
4. Use test set from step 1 to determine performance of the ensemble of $M$ classifiers (using simple majority voting)

Again note: $N$, $M$, and $F$ are parameters (in addition to $D$)

### Lab Task 3 (For Extra Practice)
Assume we have a dataset with 4 attributes ($a_1$, $a_2$, $a_3$, $a_4$) where each attribute has two possible values ($v_1$ and $v_2$) and attribute $a_5$ contains class labels with two possible values ($yes$ and $no$). Using random attribute subsets of size 2:
1. Give an example of a complete decision tree that could be generated using the random forest approach
1. Show the random attribute subset for each attribute node in the tree.